# 나이브 베이즈 분류기


## 01. 나이브 베이즈 개요

**정의**

* 나이브 베이즈(Naive Bayes)는 베이즈 정리를 기반으로 한 확률적 분류 알고리즘이다.
* 모든 특징이 서로 독립이라는 가정을 한다(→ “나이브”).

**베이즈 정리**

$P(C|X) = \frac{P(X|C) \cdot P(C)}{P(X)}$
  → 데이터 X가 주어졌을 때 클래스 C에 속할 확률을 계산한다.

- $P(C|X)$: X라는 특징이 주어졌을 때 C라는 클래스에 속할 확률 (사후 확률)
- $P(X|C)$: C 클래스가 주어졌을 때 X라는 특징이 나타날 확률 (우도)
- $P(C)$: C 클래스의 사전 확률 (클래스 발생 확률)
- $P(X)$: 특징 X가 발생할 확률


**특징**

* **장점**: 빠르고, 적은 데이터에도 강함, 고차원(특히 텍스트)에 효율적임.
* **단점**: 실제로 특징이 완전히 독립인 경우가 드물어, 상호작용이 크면 성능 저하.

**종류**

1. **Gaussian Naive Bayes**: 연속형 데이터 (정규분포 가정)
2. **Multinomial Naive Bayes**: 범주형 데이터, 텍스트(단어 빈도)
3. **Bernoulli Naive Bayes**: 이진 데이터(이벤트 발생 여부)

**주요 활용**

* 텍스트 분류(스팸 필터링, 뉴스 기사, 감정 분석)
* 의료 진단, 추천 시스템 등

**핵심**

* 단순하지만 강력하며, 특히 텍스트 데이터에 효과적임.
* 특징 독립성 가정이 현실과 다를 수 있음에 유의해야 함.

> 나이브 베이즈는 그 단순함에도 불구하고 매우 강력한 성능을 보이는 분류기 중 하나이다.
>
> 특히, 텍스트 분류와 같은 고차원의 데이터에 매우 효율적이며, 다른 복잡한 모델들에 비해 학습 속도가 빠르다는 장점이 있다.


## 02. 나이브 베이즈 분류 모델

**나이브 베이즈 분류 모델의 종류**

| **모델 이름**          | **특징**                                                                 | **적용 데이터 유형** | **사용 사례**                                |
|-----------------------|-----------------------------------------------------------------------|-----------------|-------------------------------------------|
| **Gaussian Naive Bayes** | 각 특징이 연속형 데이터로 가정하며, 가우시안(정규) 분포를 따른다고 가정한다.                       | 연속형            | Iris 데이터셋 분류, 의료 데이터 분석 등                |
| **Multinomial Naive Bayes** | 각 특징의 빈도가 중요한 경우 사용하며, 다항 분포를 가정한다. 주로 텍스트 데이터의 빈도 기반 분류에 활용된다.             | 범주형(빈도 기반)   | 뉴스 기사 분류, 텍스트 기반 감정 분석                   |
| **Bernoulli Naive Bayes**  | 각 특징이 이진형태로 존재하는 경우 사용하며, 베르누이 분포를 가정한다. 주로 이진 형태의 특성을 가진 데이터에 적합하다.       | 이진형            | 이메일 스팸 필터링, 텍스트의 긍정/부정 판별              |
| **Complement Naive Bayes** | Multinomial Naive Bayes의 변형으로, 클래스 간의 불균형을 보완하기 위해 사용된다. 클래스의 보완된 확률을 계산한다. | 범주형(빈도 기반)   | 클래스가 불균형한 텍스트 데이터 분류                      |

**특징**
- **독립 가정**: 모든 모델은 각 특징이 독립적이라고 가정한다. 이 가정 덕분에 계산이 단순해지고 빠르다.
- **조건부 확률 계산**: 각 모델은 특징과 클래스 간의 조건부 확률을 계산하여 가장 높은 확률을 가진 클래스를 선택한다.

# MultinomialNB

## 01.MultinomialNB란?

* **텍스트 분류**(스팸 필터링 등)에 자주 쓰이는 **나이브 베이즈 분류기**의 한 종류
* 입력 문장에서 **단어 빈도(count)**를 기반으로 확률을 계산
* 이름처럼 **다항분포(Multinomial distribution)** 를 기반으로 함


## 02. 다항분포

여러 개의 **범주(category)** 중에서 **n번 반복 시행**할 때, 각 범주가 나타나는 횟수를 설명하는 확률 분포

* 주사위를 10번 던졌을 때 각 눈(1~6)이 나온 횟수
* 문서에서 여러 단어가 등장한 횟수
* 설문조사에서 각 선택지가 선택된 횟수

**다항분포 확률 질량 함수 (PMF):**

$P(x_1, ..., x_k) = \frac{n!}{x_1! x_2! \cdots x_k!} \times p_1^{x_1} \times p_2^{x_2} \times \cdots \times p_k^{x_k}$

| 기호                 | 의미                          |
| ------------------ | --------------------------- |
| $n$                  | 전체 시행 횟수 (문서의 총 단어 수 등)     |
| $x_i$                 | $i$번째 단어가 등장한 횟수              |
| $p_i$                 | $i$번째 단어가 등장할 확률              |
| $\frac{n!}{x_1! x_2! \cdots x_k!}$ | 단어들이 그 순서로 배치될 경우의 수 (조합 수) |
| $p_1^{x_1} \times \cdots$        | 각 단어가 해당 횟수만큼 나올 확률         |

## 03. 나이브 베이즈 기본 공식

**베이즈 정리**를 기반:

$P(C \mid X) = \frac{P(X \mid C) \times P(C)}{P(X)}$

* $C$: 클래스 (예: 긍정/부정)
* $X$: 입력 데이터 (문서)
* $P(C)$: 클래스의 사전 확률
* $P(X \mid C)$: 클래스 $C$에서 $X$가 나올 확률 (우도)
* $P(C \mid X)$: 문서 $X$가 클래스 $C$일 확률 → 우리가 구하고 싶은 값

## 04. 나이브 가정 (단어 독립 가정)

* 각 단어는 독립적으로 등장한다고 가정:

$P(X \mid C) = P(w_1 \mid C) \times P(w_2 \mid C) \times \cdots \times P(w_n \mid C)$

이를 로그로 바꾸면:

$\log P(C \mid X) = \log P(C) + \sum_i \text{count}(w_i) \times \log P(w_i \mid C)$

* 계산 안정성과 속도 향상을 위해 **로그 연산**을 사용함

## 05. $P(w_i \mid C)$: 단어 등장 확률 계산

학습 데이터에서 **클래스 $C$에서 $w_i$가 등장한 횟수 기반으로 계산**:

$P(w_i \mid C) = \frac{\text{count}(w_i \text{ in } C) + \alpha}{\text{total\_words\_in\_C} + \alpha \times V}$

| 항목                  | 설명                         |
| ------------------- | -------------------------- |
| $\text{count}(w_i \text{ in } C)$      | 클래스 $C$에서 단어 $w_i$의 등장 횟수       |
| $\text{total\_words\_in\_C}$ | 클래스 $C$에 등장한 전체 단어 수         |
| $\alpha$                   | 라플라스 스무딩 값 (보통 1)          |
| $V$                   | 전체 어휘 크기 (vocabulary size) |

## 06. 예측 과정

> 입력 문서에 대해 각 클래스의 **log 확률을 계산**하고, 가장 높은 클래스를 선택

예: 문장 "great product" → 단어: great, product

* 긍정 클래스에서:

  $\log P(\text{긍정}) + \log P(\text{great} \mid \text{긍정}) + \log P(\text{product} \mid \text{긍정})$

* 부정 클래스에서:

  $\log P(\text{부정}) + \log P(\text{great} \mid \text{부정}) + \log P(\text{product} \mid \text{부정})$

→ 더 큰 값을 갖는 클래스가 예측 결과가 됨

In [4]:
# CounterVectorizer로 BoW(단어 빈도) 문서 벡터 생성
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

texts = [
    "I love this movie",
    "This film was amazing",
    "I hate this movie",
    "This moive was terrible"
]
labels = ["pos", "pos", "neg", "neg"]

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts)    # 단어사전 생성 및 문서/단어 희소행렬 생성
print(X_train.toarray())                     # 희소행렬을 밀집 배열로 변환해 확인

doc_vec_df = pd.DataFrame(
    X_train.toarray(),
    columns = vectorizer.get_feature_names_out()  # 컬럼명을 단어로 지정
)

doc_vec_df

[[0 0 0 1 0 1 0 1 0]
 [1 1 0 0 0 0 0 1 1]
 [0 0 1 0 0 1 0 1 0]
 [0 0 0 0 1 0 1 1 1]]


,amazing,film,hate,love,moive,movie,terrible,this,was
0,0,0,0,1,0,1,0,1,0
1,1,1,0,0,0,0,0,1,1
2,0,0,1,0,0,1,0,1,0
3,0,0,0,0,1,0,1,1,1


In [ ]:
# 멀티노미얼 나이브베이즈로 BoW 기반 감성 분류 모델 학습
from sklearn.naive_bayes import MultinomialNB  # 카운트 기반 텍스트 분류에 자주 사용하는 NB모델

clf = MultinomialNB()
clf.fit(X_train, labels)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [ ]:
# 예측
test_text = ["I love this movie"]
X_test = vectorizer.transform(test_text)  # 학습 때 만든 단어사전을 기준으로 벡터화
print(X_test.toarray())

pred = clf.predict(X_test)
pred

[[0 0 0 1 0 1 0 1 0]]


array(['pos'], dtype='<U3')

In [ ]:
# 예측 확률(predict_proba)과 클래스 순서(classes_)
proba = clf.predict_proba(X_test)  # 클래스별 확률 계산
classes = clf.classes_             # 확률 컬럼 순서에 해당하는 클래스 라벨

print(classes)
print(proba)

['neg' 'pos']
[[0.33333333 0.66666667]]


In [ ]:
# 나이브베이지의 로그 사전확률/로그 조건부확률 확인
# log 확률 : 특정 클래스 확률값 + 각 토큰별 개수 + 특정 클래스의 토큰확률

# 클래스의 사전확률 log(0.5 0.5)
class_log_prior = clf.class_log_prior_       # 클래스의 사전확률의 로그값 (log P(C))
print('클래스의 사전 확률', class_log_prior)

# 특정 클래스에서 토큰별 확률
feature_log_prob = clf.feature_log_prob_    # 클래스별 토큰 조건부확률의 로그값 (log P(token|class))
print('특정 클래스에서 토큰별 확률 :', feature_log_prob)

pd.DataFrame(
    feature_log_prob,
    columns=vectorizer.get_feature_names_out(),  # 컬럼 : 토큰(단어)
    index = clf.classes_                         # 행 : 클래스(pos/neg)
)

클래스의 사전 확률 [-0.69314718 -0.69314718]
특정 클래스에서 토큰별 확률 : [[-2.77258872 -2.77258872 -2.07944154 -2.77258872 -2.07944154 -2.07944154
  -2.07944154 -1.67397643 -2.07944154]
 [-2.07944154 -2.07944154 -2.77258872 -2.07944154 -2.77258872 -2.07944154
  -2.77258872 -1.67397643 -2.07944154]]


,amazing,film,hate,love,moive,movie,terrible,this,was
neg,-2.772589,-2.772589,-2.079442,-2.772589,-2.079442,-2.079442,-2.079442,-1.673976,-2.079442
pos,-2.079442,-2.079442,-2.772589,-2.079442,-2.772589,-2.079442,-2.772589,-1.673976,-2.079442


각 클래스에서 어떤 단어가 상대적으로 더 "그 클래스답게" 나타나는지 (조건부 확률 차이)를 해석한다.

In [9]:
X_test.toarray()

array([[0, 0, 0, 1, 0, 1, 0, 1, 0]])

In [10]:
# 나이브베이즈 로그 점수(log_score)를 직접 계산 후 softmax로 확률 변환
import torch
import torch.nn.functional as F

x_vec = X_test.toarray()[0]  # 테스트 문장의 BoW 벡터 (단어별 카운트)

log_scores = []  # 클래스별 로그 점수 저장

# 각 클래스별 log_prob을 직접 계산
for class_idx, class_name in enumerate(clf.classes_):  # 각 클래스 (neg/pos) 순회
    log_prob = class_log_prior[class_idx]              # 해당 클래스의 로그 사전확률

    for i, count in enumerate(x_vec):    # 단어별 카운트 순회
        if count > 0:                    # 등장한 단어만 반영
            log_prob += count * feature_log_prob[class_idx][i]  # 카운트x로그조건부확률 누적
    
    log_scores.append(log_prob)  # 클래스별 최종 로그 점수 저장

print(log_scores)

# 부정/긍정 확률 계산
proba = F.softmax(torch.tensor(log_scores), dim=0)  # 로그 점수를 정규화해 확률 분포로 변환
proba

[np.float64(-7.219153878051234), np.float64(-6.5260066974912885)]


tensor([0.3333, 0.6667], dtype=torch.float64)

나이브베이즈 판정은 클래스별 로그 점수(사전확률 + 단어 증거 누적)를 비교하는 것이다.  
softmax는 이를 상대 확률처럼 해석할 수 있게 된다.

### 뉴스그룹 토픽 분류

In [11]:
from sklearn.datasets import fetch_20newsgroups    # 20개 뉴스그룹 텍스트 분류 데이터셋

newsgroup = fetch_20newsgroups(subset='all')  # 전체 데이터(학습 + 테스트) 로드
print(newsgroup.data[0])                      # 첫 번째 문서 확인
print(newsgroup.target[0])                    # 첫 번째 문서의 클래스 인덱스 확인
print(newsgroup.target_names)                 # 클래스 인덱스 -> 클래스 이름 목록 확인

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!


10
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.

In [12]:
# 20 Newsgroups 텍스트를 TF-IDF로 벡터화 후 학습/테스트 분리
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = newsgroup.data     # 문서(텍스트) 리스트
y = newsgroup.target   # 문서별 클래스 라벨(정수)

vectorizer = TfidfVectorizer(stop_words='english')  # 영어 불용어 제거
X_transformed = vectorizer.fit_transform(X)         # 문서 -> TF-IDF 희소행렬로 변환
print(X_transformed.shape)                          # (문서 수, 단어 수) 크기 확인

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)  # 학습/테스트 데이터 분리
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(18846, 173451)
(15076, 173451) (15076,)
(3770, 173451) (3770,)


In [13]:
# 멀티노미얼 나이브베이즈로 TF-IDF 기반 뉴스그룹 분류 모델 학습
model = MultinomialNB()
model.fit(X_train, y_train)

,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [14]:
# 뉴스그룹 분류 모델 평가
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)
print('Accuracy :', accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred, target_names = newsgroup.target_names))  # 클래스 이름 매핑 적용

Accuracy : 0.8777188328912466
                          precision    recall  f1-score   support

             alt.atheism       0.85      0.85      0.85       151
           comp.graphics       0.88      0.84      0.86       202
 comp.os.ms-windows.misc       0.86      0.85      0.85       195
comp.sys.ibm.pc.hardware       0.64      0.85      0.73       183
   comp.sys.mac.hardware       0.94      0.87      0.90       205
          comp.windows.x       0.95      0.85      0.90       215
            misc.forsale       0.93      0.72      0.81       193
               rec.autos       0.91      0.94      0.92       196
         rec.motorcycles       0.89      0.95      0.92       168
      rec.sport.baseball       0.95      0.95      0.95       211
        rec.sport.hockey       0.90      0.99      0.94       198
               sci.crypt       0.91      0.97      0.93       201
         sci.electronics       0.92      0.82      0.86       202
                 sci.med       0.97      0.93